In [1]:
import scanpy as sc
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
main_dir = "/projects/robson-lab/research/endometriosis/"
sample_id = "Endometriosis-revision-3"

In [3]:
#load in dataset
mye = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Myeloid/{sample_id}-mye-final-cc-20220105.h5ad")
lym = sc.read(f"{main_dir}/data/h5ad/{sample_id}/TNK/{sample_id}-lym-final-cc-20220105.h5ad")
epi = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Epithelial/{sample_id}-epi-final-cc-20220119.h5ad")
stro = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Stromal/{sample_id}-stro-final-cc-20220105.h5ad")
endo = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Endothelial/{sample_id}-endo-final-cc-20220105.h5ad")

In [4]:
lym.obs["subtypes"] = lym.obs.subtypes.map({
    "T$_{EM}$": "Tem",
    "CD8 T$_{RM}$":"CD8Trm",
    "CD4 T$_{RM}$":"CD4Trm",
    "T$_N$/T$_{CM}$":"Tncm",
    "T$_{Reg}$":"Treg",
    "NK1":"NK1",
    "NK2":"NK2",
    "NK3":"NK3",
    "pNK":"pNK",
    "CTL":"CTL",
    "CD8 MAIT":"CD8 MAIT",
    "plasma":"plasma",
    "B cell":"B cell",
    "ILC":"ILC"
})

mye.obs["subtypes"] = mye.obs.subtypes.map({
    "monocytes-CD16-":"mono (CD16-)",
    "M$\Phi$4-infiltrated":"M4",
    "M$\Phi$1-LYVE1":"M1",
    "cDC2":"cDC2",
    "M$\Phi$3-APOE":"M3",
    "M$\Phi$5-activated":"M5",
    "pre-cDC2":"precDC2",
    "monocytes-CD16+":"mono (CD16+)",
    "mast cells":"mast cells",
    "DC3":"DC3",
    "M$\Phi$2-peritoneal":"M2",
    "cDC1":"cDC1",
    "mDC":"mDC",
    "pDC":"pDC",
    "granulocytes":"granulocytes",
})

In [5]:
#adatas = [mye, lym, epi, stro, endo]
adatas = [epi]
for adata in adatas:
    subtypes_list = (adata.obs.subtypes.value_counts() > 500).to_frame("min500cells")
    subset_to_keep = subtypes_list[subtypes_list.min500cells == True].index.tolist()
    subset = adata[adata.obs.subtypes.isin(subset_to_keep)].copy()
    N = len(subset_to_keep)
    
    sub_dir = "DEG/edgeR-input"
    output_dir = f"{main_dir}analysis/{sample_id}/{sub_dir}"
    
    for item in range(0,N):
        dataset = subset[subset.obs.subtypes == subset_to_keep[item]]
        counts_matrix = pd.DataFrame(dataset.layers["raw"].todense(), 
                                     index=dataset.obs_names,columns=dataset.var_names).T
        metadata = dataset.obs.sample_type_rename.to_frame("group")

        #export data
        metadata.to_csv(f"{output_dir}/{subset_to_keep[item]}-metadata.csv")
        counts_matrix.to_csv(f"{output_dir}/{subset_to_keep[item]}-counts.csv")

/home/tany/.conda/envs/endometriosis/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [9]:
adatas = [mye, lym, epi, stro, endo]
subtypes = []
for adata in adatas:
    subtypes_list = (adata.obs.subtypes.value_counts() > 500).to_frame("min500cells")
    subset_to_keep = subtypes_list[subtypes_list.min500cells == True].index.tolist()
    subtypes+=subset_to_keep


sub_dir = "DEG/edgeR-input/"
output_dir = f"{main_dir}analysis/{sample_id}/{sub_dir}"
pd.DataFrame(subtypes,columns=["0"]).to_csv(f"{output_dir}/subtypelist.csv")

---